# Model 35


|Markets|    X  |  Y  |  G |  W1 |  W2  |     CONS1 |  CONS2 |  GOVT|
|-------|-------|-----|----|-----|------|-----------|--------|------|
|  PX   |  100  |     |    | -70 | -30  |           |        |      |     
|  PY   |       |100  |    | -30 | -70  |           |        |      |     
|  PG   |       |     | 50 |     |      |           |        |   -50|      
|  PL   |  -50  |-30  |-20 |     |      |        50 |     50 |      |      
|  PK   |  -30  |-50  |-20 |     |      |        50 |     50 |      |       
|  TAX  |  -20  |-20  |-10 |     |      |           |        |    50|       
|  PW1  |       |     |    |  125|      |      -125 |        |      |      
|  PW2  |       |     |    |     |  125 |           |   -125 |      |      
|  PG1  |       |     |    |  -25|      |        25 |        |      |      
|  PG2  |       |     |    |     |  -25 |           |     25 |      |      


In [2]:
using MPSGE

In [5]:
M35 = MPSGEModel()

@parameter(M35, VG1, 1, description = "Preference index for public goods for consumer 1")

@sectors(M35, begin
	X, (description = "Activity level for sector X",)
	Y, (description = "Activity level for sector Y",)
	G, (description = "Activity level for sector G  (public provision)",)
	W1, (description = "Activity level for sector W1 (consumer 1 welfare index)",)
	W2, (description = "Activity level for sector W2 (consumer 2 welfare index)",)
end)

@commodities(M35, begin
	PX, (description = "Price index for commodity X",)
	PY, (description = "Price index for commodity Y",)
	PG, (description = "Price index for commodity G(marginal cost of public output)",)
	PL, (description = "Price index for primary factor L (net of tax)",)
	PK, (description = "Price index for primary factor K",)
	PW1, (description = "Price index for welfare (consumer 1)",)
	PW2, (description = "Price index for welfare (consumer 2)",)
	PG1, (description = "Private valuation of the public good (consumer 1)",)
	PG2, (description = "Private valuation of the public good (consumer 2)",)
end)

@consumers(M35, begin
	CONS1, (description = "Consumer 1",)
	CONS2, (description = "Consumer 2",)
	GOVT, (description = "Government",)
end)

@auxiliaries(M35, begin
	LGP, (description = "Level of government provision",)
    TAX, (description = "Uniform value-added tax rate",)
end)


@production(M35, X, [s=1,t=0], begin
    @output(PX, 100, t)
    @input(PL, 50, s, taxes=[Tax(GOVT, TAX)], reference_price = 1.25)
    @input(PK, 30, s, taxes=[Tax(GOVT, TAX)], reference_price = 1.25)
end)

@production(M35, Y, [s=1,t=0], begin
    @output(PY, 100, t)
    @input(PL, 30, s, taxes=[Tax(GOVT, TAX)], reference_price = 1.25)
    @input(PK, 50, s, taxes=[Tax(GOVT, TAX)], reference_price = 1.25)
end)

@production(M35, G, [s=1,t=0], begin
    @output(PG, 50, t)
    @input(PL, 20, s, taxes=[Tax(GOVT, TAX)], reference_price = 1.25)
    @input(PK, 20, s, taxes=[Tax(GOVT, TAX)], reference_price = 1.25)
end)

@production(M35, W1, [s=1,t=0], begin
    @output(PW1, 125, t)
    @input(PX, 70, s)
    @input(PY, 30, s)
    @input(PG1, 50*VG1, s, reference_price = .5)
end)

@production(M35, W2, [s=1,t=0], begin
    @output(PW2, 125, t)
    @input(PX, 30, s)
    @input(PY, 70, s)
    @input(PG2, 50, s, reference_price = .5)
end)

@demand(M35, GOVT, begin
    @final_demand(PG, 50)
end,begin
    
end)


@demand(M35, CONS1, begin
    @final_demand(PW1, 125)
    end,begin
    @endowment(PL, 50)
    @endowment(PK, 50)
    @endowment(PG1, 50*LGP)
end)

@demand(M35, CONS2, begin
    @final_demand(PW2, 125)
end,begin
    @endowment(PL,50)
    @endowment(PK, 50)
    @endowment(PG2, 50*LGP)
end)

@aux_constraint(M35, LGP,
    LGP - G
);

@aux_constraint(M35, TAX,
    PG - PG1 - PG2
);

# Benchmark

In [6]:
set_start_value(TAX, .25)
set_start_value(LGP, 1)
set_start_value(PG1, .5)
set_start_value(PG2, .5)

solve!(M35; cumulative_iteration_limit =0)

generate_report(M35)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_59A2.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 18

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (zero_profit[X)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Solver Status: LOCALLY_SOLVED
Model Status: FEASIBLE_POINT

Default price normalization using income for CONS2 - This value is fixed. Unfix with unfix(CONS2).

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,1.0,0.0
2,Y,1.0,0.0
3,G,1.0,0.0
4,W2,1.0,0.0
5,W1,1.0,0.0
6,PW2,1.0,0.0
7,PL,1.0,0.0
8,PG,1.0,0.0
9,PG2,0.5,0.0


# Counterfactual

In [7]:
set_value!(VG1, 2)

solve!(M35)
generate_report(M35)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_A012.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 18

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.0991e+00             0.0e+00 (zero_profit[W1)
    1     6     0    18 1.0882e+00  1.7e-01    0.0e+00 (zero_profit[W1)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     7     2 1.0882e+00           I 0.0e+00 9.4e-01 (zero_profit[W1)
    1     1     8     3 3.2465e+00  1.0e+00 SM 0.0e+00 1.9e+00 (market_clearanc)
    2     1     9     4 1.2235e+00  1.0e+00 SO 0.0e+00 5.7e-01 (market_clearanc)
    3     1    10     5 1.1085e-02  1.0e+00 SO 0.0e+00 7.5e-03 (market_clearanc)
    4     1    11     6 3.5221e-07  1.0e+00 SO 0.0e+0

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,0.909091,4.13085e-8
2,Y,0.909091,3.901e-8
3,G,1.36364,2.00763e-8
4,W2,0.985883,-4.85204e-9
5,W1,0.825964,-2.37341e-9
6,PW2,1.01432,1.22518e-7
7,PL,1.0,1.71769e-9
8,PG,1.1,-2.98256e-7
9,PG2,0.366667,3.13641e-9
